In [21]:
import os
import polars as pl


In [22]:
def target_encoding_ranking_polars(df, columna):
    try:
        # 1. Calcular frecuencias y ranking (igual que antes)
        freq_ranking = (
            df
            .group_by(columna)
            .agg(pl.len().alias('count'))
            .sort('count', descending=True)
            .with_row_index('ranking', offset=1)
            .with_columns([
                (pl.col('ranking') + (pl.col('count') / 10000.0))
                .alias('encoding_value')
            ])
        )
        
        # 2. Crear diccionario de mapeo
        encoding_map = dict(
            zip(
                freq_ranking[columna].to_list(),
                freq_ranking['encoding_value'].to_list()
            )
        )
        
        return encoding_map
        
    except Exception as e:
        print(f"Error en target_encoding_ranking_polars: {e}")
        return {}

def one_hot_encoding_polars(df, columna):
    try:
        # Obtener valores únicos de la columna
        valores_unicos = df[columna].unique().to_list()
        
        # Crear columnas one-hot
        expresiones_onehot = []
        for valor in valores_unicos:
            # Limpiar nombre de columna (remover espacios y caracteres especiales)
            nombre_columna = f"{columna}_{valor}".replace(" ", "_").replace("(", "").replace(")", "").replace("-", "_")
            
            # Crear expresión binaria
            expresion = (pl.col(columna) == valor).cast(pl.Int8).alias(nombre_columna)
            expresiones_onehot.append(expresion)
        
        # Aplicar todas las expresiones one-hot Y eliminar columna original
        df_onehot = df.with_columns(expresiones_onehot).drop(columna)
        
        return df_onehot
        
    except Exception as e:
        print(f"Error en one_hot_encoding_polars: {e}")
        return df

def normalizar_columna_polars(df, columna, metodo='minmax'):
    """
    Normaliza una columna numérica usando MinMax o Standard Scaler.
    
    Args:
        df (pl.DataFrame): DataFrame de Polars
        columna (str): Nombre de la columna numérica a normalizar
        metodo (str): 'minmax' o 'standard'
    
    Returns:
        pl.DataFrame: DataFrame con la columna normalizada (reemplaza la original)
        
    Ejemplos:
        MinMax: valores entre 0 y 1
        Standard: media=0, desviación=1
    """
    try:
        if metodo == 'minmax':
            # MinMax: (valor - min) / (max - min)
            df_normalizado = df.with_columns([
                (
                    (pl.col(columna) - pl.col(columna).min()) / 
                    (pl.col(columna).max() - pl.col(columna).min())
                ).alias(f"{columna}")
            ])
            
        elif metodo == 'standard':
            # Standard: (valor - media) / desviación
            df_normalizado = df.with_columns([
                (
                    (pl.col(columna) - pl.col(columna).mean()) / 
                    pl.col(columna).std()
                ).alias(f"{columna}")
            ])
            
        else:
            print(f"Método '{metodo}' no reconocido. Usa 'minmax' o 'standard'")
            return df
            
        return df_normalizado
        
    except Exception as e:
        print(f"Error en normalizar_columna_polars: {e}")
        return df

In [23]:
# Cargar dataset original
df_medicamentos = pl.read_parquet('./data/medicamentos_preprocesados.parquet')

print(f"\n📊 Dataset cargado:")
print(f"   - Shape: {df_medicamentos.shape}")
print(f"   - Columnas: {df_medicamentos.columns}")



📊 Dataset cargado:
   - Shape: (248635, 17)
   - Columnas: ['CUM', 'EXPEDIENTE CUM', 'CONSECUTIVO', 'ESTADO REGISTRO', 'CANTIDAD CUM', 'ESTADO CUM', 'MUESTRA MÉDICA', 'UNIDAD', 'ATC', 'DESCRIPCIÓN_ATC', 'VÍA ADMINISTRACIÓN', 'CONCENTRACIÓN', 'PRINCIPIO ACTIVO', 'UNIDAD MEDIDA', 'CANTIDAD', 'UNIDAD REFERENCIA', 'FORMA FARMACÉUTICA']


In [24]:
print("\n💾 Guardando dataset original completo con CUM...")
df_medicamentos.select([
    'CUM',                 # ⚠️ CÓDIGO ÚNICO DEL MEDICAMENTO - NUNCA ELIMINAR
    'ATC',                 # 2,419 únicos - Código farmacológico
    'VÍA ADMINISTRACIÓN',  # 51 únicos - Vía de administración 
    'FORMA FARMACÉUTICA',  # 112 únicos - Presentación física
    'PRINCIPIO ACTIVO',    # 20,321 únicos - Ingrediente activo
    'ESTADO REGISTRO',     # 14 únicos - Estado regulatorio
    'ESTADO CUM',          # 2 únicos - Estado del CUM
    'MUESTRA MÉDICA',      # 2 únicos - Si es muestra
    'CANTIDAD CUM',        # Numérica - Cantidad presentación
    'UNIDAD',              # 13 únicos - U, ml, g   
    'CANTIDAD',            # Numérica - Dosis
    'UNIDAD MEDIDA',       # 180 únicos - mg, g, mcg
    'EXPEDIENTE CUM'       # 34,653 únicos - Expediente (parte de la familia principal del medicamento)
]).write_parquet('./data/medicamentos_train_original.parquet')

print("✅ Dataset original con CUM guardado")


💾 Guardando dataset original completo con CUM...
✅ Dataset original con CUM guardado


In [25]:
print("\n🔨 Creando dataset de trabajo PRESERVANDO CUM...")

# ⚠️ INCLUIR CUM EXPLÍCITAMENTE AL INICIO
df_medicamentos_train = df_medicamentos.select([
    'CUM',                 # ⚠️ PRESERVAR CUM - IDENTIFICADOR PRINCIPAL
    'ATC',                 # 2,419 únicos - Código farmacológico
    'VÍA ADMINISTRACIÓN',  # 51 únicos - Vía de administración 
    'FORMA FARMACÉUTICA',  # 112 únicos - Presentación física
    'PRINCIPIO ACTIVO',    # 20,321 únicos - Ingrediente activo
    'ESTADO REGISTRO',     # 14 únicos - Estado regulatorio
    'ESTADO CUM',          # 2 únicos - Estado del CUM
    'MUESTRA MÉDICA',      # 2 únicos - Si es muestra
    'CANTIDAD CUM',        # Numérica - Cantidad presentación
    'UNIDAD',              # 13 únicos - U, ml, g   
    'CANTIDAD',            # Numérica - Dosis
    'UNIDAD MEDIDA',       # 180 únicos - mg, g, mcg
    'EXPEDIENTE CUM'       # 34,653 únicos - Expediente
])


df_medicamentos_train = df_medicamentos_train.with_columns(
    pl.col('EXPEDIENTE CUM').cast(pl.Utf8)
)


print(f"✅ Dataset de trabajo creado:")
print(f"   - Shape: {df_medicamentos_train.shape}")


🔨 Creando dataset de trabajo PRESERVANDO CUM...
✅ Dataset de trabajo creado:
   - Shape: (248635, 13)


In [26]:
atc_map = target_encoding_ranking_polars(df_medicamentos, 'ATC')
df_medicamentos_train = df_medicamentos_train.with_columns(
    pl.col('ATC').replace(atc_map).cast(pl.Float64).alias('ATC')
)

via_map = target_encoding_ranking_polars(df_medicamentos_train, 'VÍA ADMINISTRACIÓN')
df_medicamentos_train = df_medicamentos_train.with_columns(
    pl.col('VÍA ADMINISTRACIÓN').replace(via_map).cast(pl.Float64).alias('VÍA ADMINISTRACIÓN')
)

forma_map = target_encoding_ranking_polars(df_medicamentos_train, 'FORMA FARMACÉUTICA')
df_medicamentos_train = df_medicamentos_train.with_columns(
    pl.col('FORMA FARMACÉUTICA').replace(forma_map).cast(pl.Float64).alias('FORMA FARMACÉUTICA')
)

principio_map = target_encoding_ranking_polars(df_medicamentos, 'PRINCIPIO ACTIVO')
df_medicamentos_train = df_medicamentos_train.with_columns(
    pl.col('PRINCIPIO ACTIVO').replace(principio_map).cast(pl.Float64).alias('PRINCIPIO ACTIVO')
)

df_medicamentos_train = df_medicamentos_train.with_columns(
    pl.col('ESTADO REGISTRO').map_elements(lambda x: 1 if x == "Vigente" else 0, return_dtype=pl.Int8),
    pl.col('ESTADO CUM').map_elements(lambda x: 1 if x == "Activo" else 0, return_dtype=pl.Int8),
    pl.col('MUESTRA MÉDICA').map_elements(lambda x: 1 if x == "No" else 0, return_dtype=pl.Int8),
)

df_medicamentos_train = normalizar_columna_polars(df_medicamentos_train, 'CANTIDAD CUM', 'standard')
df_medicamentos_train = normalizar_columna_polars(df_medicamentos_train, 'CANTIDAD', 'standard')

unidad_map = target_encoding_ranking_polars(df_medicamentos, 'UNIDAD')
df_medicamentos_train = df_medicamentos_train.with_columns(
    pl.col('UNIDAD').replace(unidad_map).cast(pl.Float64).alias('UNIDAD')
)

unimed_map = target_encoding_ranking_polars(df_medicamentos, 'UNIDAD MEDIDA')
df_medicamentos_train = df_medicamentos_train.with_columns(
    pl.col('UNIDAD MEDIDA').replace(unimed_map).cast(pl.Float64).alias('UNIDAD MEDIDA')
)

expediente_map = target_encoding_ranking_polars(df_medicamentos, 'EXPEDIENTE CUM')
df_medicamentos_train = df_medicamentos_train.with_columns(
    pl.col('EXPEDIENTE CUM').replace(expediente_map).cast(pl.Float64).alias('EXPEDIENTE CUM')
)

# df_medicamentos_train = df_medicamentos_train.with_columns([
#     (
#         (pl.col('ESTADO REGISTRO') == 1) &
#         (pl.col('ESTADO CUM') == 1) &
#         (pl.col('MUESTRA MÉDICA') == 1)
#     ).cast(pl.Int8).alias('ES_VALIDO')
# ])

df_medicamentos_train.head(5)

CUM,ATC,VÍA ADMINISTRACIÓN,FORMA FARMACÉUTICA,PRINCIPIO ACTIVO,ESTADO REGISTRO,ESTADO CUM,MUESTRA MÉDICA,CANTIDAD CUM,UNIDAD,CANTIDAD,UNIDAD MEDIDA,EXPEDIENTE CUM
str,f64,f64,f64,f64,i8,i8,i8,f64,f64,f64,f64,f64
"""20167292-1""",1708.0006,18.3064,7.4395,14035.0002,1,1,1,-0.071174,22.4076,-0.00413,19.1932,15511.0004
"""20015204-5""",9.2139,18.3064,60.031,2411.0016,1,1,1,-0.091356,22.4076,-0.004135,19.1932,35.0224
"""20061748-3""",7.284,5.8494,21.2678,218.0152,1,0,1,-0.091356,22.4076,-0.004135,6.2562,28.0243
"""20061746-8""",7.284,3.8369,21.2678,224.015,1,0,0,-0.091356,22.4076,-0.004135,19.1932,27.0252
"""20009806-9""",2.8577,18.3064,5.8483,12.1041,0,1,1,-0.057256,22.4076,-0.004135,19.1932,742.004


In [27]:
print(f"\n📊 ESTRUCTURA DEL DATASET FINAL:")
print(f"   - Shape: {df_medicamentos_train.shape}")
print(f"   - Columnas totales: {len(df_medicamentos_train.columns)}")
print(f"   - CUM presente: {'CUM' in df_medicamentos_train.columns}")
# print(f"   - ES_VALIDO presente: {'ES_VALIDO' in df_medicamentos_train.columns}")
print(f"   - Primera columna: {df_medicamentos_train.columns[0]}")



📊 ESTRUCTURA DEL DATASET FINAL:
   - Shape: (248635, 13)
   - Columnas totales: 13
   - CUM presente: True
   - Primera columna: CUM


In [28]:
df_medicamentos_train.write_parquet('./data/medicamentos_train_preprocesados.parquet')

In [29]:
df_medicamentos_train

CUM,ATC,VÍA ADMINISTRACIÓN,FORMA FARMACÉUTICA,PRINCIPIO ACTIVO,ESTADO REGISTRO,ESTADO CUM,MUESTRA MÉDICA,CANTIDAD CUM,UNIDAD,CANTIDAD,UNIDAD MEDIDA,EXPEDIENTE CUM
str,f64,f64,f64,f64,i8,i8,i8,f64,f64,f64,f64,f64
"""20167292-1""",1708.0006,18.3064,7.4395,14035.0002,1,1,1,-0.071174,22.4076,-0.00413,19.1932,15511.0004
"""20015204-5""",9.2139,18.3064,60.031,2411.0016,1,1,1,-0.091356,22.4076,-0.004135,19.1932,35.0224
"""20061748-3""",7.284,5.8494,21.2678,218.0152,1,0,1,-0.091356,22.4076,-0.004135,6.2562,28.0243
"""20061746-8""",7.284,3.8369,21.2678,224.015,1,0,0,-0.091356,22.4076,-0.004135,19.1932,27.0252
"""20009806-9""",2.8577,18.3064,5.8483,12.1041,0,1,1,-0.057256,22.4076,-0.004135,19.1932,742.004
…,…,…,…,…,…,…,…,…,…,…,…,…
"""20048292-2""",17.1415,18.3064,4.754,2779.0014,0,0,1,-0.085093,22.4076,-0.004135,19.1932,3537.0014
"""19927508-2""",573.0075,18.3064,8.6413,41.0548,0,0,1,-0.00854,22.4076,-0.004135,6.2562,26146.0002
"""19952712-1""",1430.0011,18.3064,5.7013,3453.0011,1,1,1,-0.022459,22.4076,-0.004135,19.1932,9480.0006
